# Scraping Safety Data - City of Toronto Police

Because this is GEOJSON data, I could not use Beautifulsoup4. So, the processes below is really to import and convert to a pandas dataframe. Then, I can explore these data.

In [3]:
import osmnx  # to import and manage road netwrok data
import geopandas  # needed get background data
import requests  # for making standard html requests
import pandas  # premier library for data organization
import numpy  # for data manipulation
import rasterstats # to interpolate raster data
from random import shuffle # for managing psedoabsences

URL_Robbery = 'https://opendata.arcgis.com/datasets/9115accc55f24938b4eb573dd222c33b_0.geojson'
URL_Pedestrian = 'https://opendata.arcgis.com/datasets/1e8a71c533fb4b0aa522cf1b1236bee7_0.geojson'
page_robbery = requests.get(URL_Robbery).json()
page_pedestrian = requests.get(URL_Pedestrian).json()

pedestrian_json_df = pandas.json_normalize(page_pedestrian['features'])
robbery_json_df = pandas.json_normalize(page_robbery['features'])

Interested in the number of features and which columns I should keep.

In [4]:
list(pedestrian_json_df)
len(pedestrian_json_df)

6484

In [5]:
list(robbery_json_df)
len(robbery_json_df)

21543

Strange, but each column has a properties. prefix. Looks weird, so I am going to remove it

In [6]:
pedestrian_json_df.columns = pedestrian_json_df.columns.str.lstrip('properties.')
robbery_json_df.columns = robbery_json_df.columns.str.lstrip('properties.')

Focusing on features that I believe are predictive.

In [7]:
pedestrian_df = pedestrian_json_df.iloc[:, numpy.r_[3,5,6, 10, 14,15, 19, 20]]
robbery_df = robbery_json_df.iloc[:, numpy.r_[1:3, 9, 10, 11, 12, 13, 14, 5, 8, 22, 23, 25, 26]]

I will subset the data. For the robbery data, I only want the mugging data because this is most applicable for pedestrians

In [8]:
robbery_df_mugging = robbery_df.loc[robbery_df['ffence'] == 'Robbery - Mugging']  # 6847 rows
robbery_df_mugging_outside = robbery_df_mugging.loc[robbery_df_mugging['misetype'] == 'Outside']  # 5141 rows

Some features are unbalanced. I will remove accident locations with few observations (<10)


In [9]:
pedestrian_df_road = pedestrian_df.loc[pedestrian_df['ROAD_CLASS'].isin(['Major Arterial', 'Minor Arterial',
                                                                         'Collector', 'Local',
                                                                         'Expressway'])]  # 4986 rows

I will need to create geaodataframes many times, so a function will be best to make the code cleaner.

In [10]:
def create_gdf(df, Longitude, Latitude, projection):
    return geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df[Longitude], df[Latitude]),
                                  crs=projection)

Need as spatial data to measure distance to features later.

In [11]:
robbery_points_gdf_2958 = create_gdf(df=robbery_df_mugging_outside,
                                     Latitude="Lat",
                                     Longitude="Long",
                                     projection="EPSG:4326")

robbery_points_gdf_4326 = create_gdf(df=robbery_df_mugging_outside,
                                     Latitude="Lat",
                                     Longitude="Long",
                                     projection="EPSG:4326")
robbery_points_gdf_2958 = robbery_points_gdf_2958.to_crs(epsg = 2958) 
pedestrian_points_gdf_2958 = create_gdf(df=pedestrian_df_road,
                                     Latitude="LATITUDE",
                                     Longitude="LONGITUDE",
                                     projection="EPSG:4326")

pedestrian_points_gdf_4326 = create_gdf(df=pedestrian_df_road,
                                        Latitude="LATITUDE",
                                        Longitude="LONGITUDE",
                                     projection="EPSG:4326")
pedestrian_points_gdf_2958 = pedestrian_points_gdf_2958.to_crs(epsg = 2958)  

Need consistency in labelling for future analyses

In [12]:
robbery_points_gdf_2958 = robbery_points_gdf_2958.rename(columns={"Long": "Longitude", "Lat": "Latitude"})
robbery_points_gdf_4326 = robbery_points_gdf_4326.rename(columns={"Long": "Longitude", "Lat": "Latitude"})
pedestrian_points_gdf_2958 = pedestrian_points_gdf_2958.rename(columns={"LONGITUDE": "Longitude", "LATITUDE": "Latitude"})
pedestrian_points_gdf_4326 = pedestrian_points_gdf_4326.rename(columns={"LONGITUDE": "Longitude", "LATITUDE": "Latitude"})

It will be easier to re-run making the psedoabsences if I make a function considering that I might extend to the GTA and not just Old Toronto.

In [13]:
def get_pseudoabsence_data(lat_min, lat_max, lon_min, lon_max, number_of_points):
    numpy.random.seed(13)
    pseudoabsence_latitudes = numpy.random.uniform(lat_min, lat_max, number_of_points)
    pseudoabsence_longitudes = numpy.random.uniform(lon_min, lon_max, number_of_points)
    pseudoabsences = pandas.DataFrame(numpy.transpose(numpy.array([pseudoabsence_longitudes, pseudoabsence_latitudes])),
                                      columns=['Longitude', 'Latitude'])
    return (pseudoabsences)

Some of the background points will fall in the lake, so multipling by 4 to make sure I have enough falling on land. Then, I will convert to a spaital layer for later measuring distance to feature, which I will include in my predictive model.

In [14]:
robbery_backgroundpoints = get_pseudoabsence_data(lat_min=robbery_points_gdf_4326['Latitude'].min(),
                                                  lat_max=robbery_points_gdf_4326['Latitude'].max(),
                                                  lon_min=robbery_points_gdf_4326['Longitude'].min(),
                                                  lon_max=robbery_points_gdf_4326['Longitude'].max(),
                                                  number_of_points=len(robbery_points_gdf_4326) * 4)

robbery_backgroundpoints_gdf_2958 = create_gdf(df=robbery_backgroundpoints,
                                               Latitude="Latitude",
                                               Longitude="Longitude",
                                               projection="EPSG:4326")
robbery_backgroundpoints_gdf_4326 = create_gdf(df=robbery_backgroundpoints,
                                               Latitude="Latitude",
                                               Longitude="Longitude",
                                               projection="EPSG:4326")

robbery_backgroundpoints_gdf_2958 = robbery_backgroundpoints_gdf_2958.to_crs(epsg = 2958) 

pedestrian_backgroundpoints = get_pseudoabsence_data(lat_min=pedestrian_points_gdf_4326['Latitude'].min(),
                                                  lat_max=pedestrian_points_gdf_4326['Latitude'].max(),
                                                  lon_min=pedestrian_points_gdf_4326['Longitude'].min(),
                                                  lon_max=pedestrian_points_gdf_4326['Longitude'].max(),
                                                  number_of_points=len(pedestrian_points_gdf_4326) * 4)

pedestrian_backgroundpoints_gdf_2958 = create_gdf(df=pedestrian_backgroundpoints,
                                               Latitude="Latitude",
                                               Longitude="Longitude",
                                               projection="EPSG:4326")
pedestrian_backgroundpoints_gdf_4326 = create_gdf(df=pedestrian_backgroundpoints,
                                               Latitude="Latitude",
                                               Longitude="Longitude",
                                               projection="EPSG:4326")

pedestrian_backgroundpoints_gdf_2958 = pedestrian_backgroundpoints_gdf_2958.to_crs(epsg = 2958) 

Importing location Data to Measure Proximity transporation data to later measure distance to feature for regression/random forest need a UTM projection for distance to feature.


In [15]:
place_name = 'Old Toronto, Toronto, Golden Horseshoe, Ontario, Canada'

selected_tags = {
    'highway': True}

stations_OldToronto = osmnx.pois_from_place(place=place_name, tags=selected_tags)
features_location_names = [
    'crossing', 'give_way', 'stop',
    'traffic_signals', 'turning_loop', 'speed_camera']

Dropping some fetaures that I do not need some of them are polygons.

In [85]:
stations_OldToronto_4326 = stations_OldToronto[stations_OldToronto.highway.isin(features_location_names)]
stations_OldToronto_4326 = stations_OldToronto_4326.iloc[:, numpy.r_[0, 1, 4]]

In [86]:
stations_OldToronto_4326

,osmid,geometry,highway
20380905,20380905,POINT (-79.41168 43.71211),traffic_signals
20380929,20380929,POINT (-79.39758 43.67721),traffic_signals
20380931,20380931,POINT (-79.43902 43.66846),traffic_signals
20380933,20380933,POINT (-79.45122 43.66574),traffic_signals
20380934,20380934,POINT (-79.46378 43.66364),traffic_signals
...,...,...,...
7598237115,7598237115,POINT (-79.38698 43.68947),stop
7599063699,7599063699,POINT (-79.35499 43.64827),traffic_signals
7599063700,7599063700,POINT (-79.35463 43.64807),traffic_signals
7599063701,7599063701,POINT (-79.35657 43.64943),traffic_signals


Converting to other projections gives me errors, so the gemteory tab is in ESPG 2958, but it read in as ESPG 4326 I will need to covert this to a etract the x y data convert to a dataframe, then reproject.

In [87]:
stations_OldToronto_4326['x'] = stations_OldToronto_4326['geometry'].x
stations_OldToronto_4326['y'] = stations_OldToronto_4326['geometry'].y
stations_OldToronto_4326 = pandas.DataFrame(stations_OldToronto_4326)

In [88]:
stations_OldToronto_4326

,osmid,geometry,highway,x,y
20380905,20380905,POINT (-79.41168 43.71211),traffic_signals,-79.411685,43.712112
20380929,20380929,POINT (-79.39758 43.67721),traffic_signals,-79.397579,43.677207
20380931,20380931,POINT (-79.43902 43.66846),traffic_signals,-79.439018,43.668458
20380933,20380933,POINT (-79.45122 43.66574),traffic_signals,-79.451220,43.665739
20380934,20380934,POINT (-79.46378 43.66364),traffic_signals,-79.463781,43.663637
...,...,...,...,...,...
7598237115,7598237115,POINT (-79.38698 43.68947),stop,-79.386975,43.689471
7599063699,7599063699,POINT (-79.35499 43.64827),traffic_signals,-79.354992,43.648266
7599063700,7599063700,POINT (-79.35463 43.64807),traffic_signals,-79.354631,43.648065
7599063701,7599063701,POINT (-79.35657 43.64943),traffic_signals,-79.356570,43.649431


Converting the node data to a geopanda dataframe so that I can extract elevation data.

In [89]:
stations_OldToronto_4326 = create_gdf(df=stations_OldToronto_4326,
                                      Latitude="y",
                                      Longitude="x",
                                      projection="EPSG:4326")
stations_OldToronto_2958 = create_gdf(df=stations_OldToronto_4326,
                                      Latitude="y",
                                      Longitude="x",
                                      projection="EPSG:4326")
stations_OldToronto_2958 = stations_OldToronto_4326.to_crs(epsg=2958) 

stations_OldToronto_2958 = stations_OldToronto_2958.iloc[:, numpy.r_[0, 1, 2]]
stations_OldToronto_2958['x'] = stations_OldToronto_2958['geometry'].x
stations_OldToronto_2958['y'] = stations_OldToronto_2958['geometry'].y

In [90]:
stations_OldToronto_2958

,osmid,geometry,highway,x,y
20380905,20380905,POINT (627956.636 4841124.276),traffic_signals,627956.635724,4.841124e+06
20380929,20380929,POINT (629167.988 4837269.518),traffic_signals,629167.988125,4.837270e+06
20380931,20380931,POINT (625845.870 4836234.148),traffic_signals,625845.869579,4.836234e+06
20380933,20380933,POINT (624867.775 4835913.706),traffic_signals,624867.774586,4.835914e+06
20380934,20380934,POINT (623859.425 4835661.379),traffic_signals,623859.425244,4.835661e+06
...,...,...,...,...,...
7598237115,7598237115,POINT (629996.218 4838648.193),stop,629996.217563,4.838648e+06
7599063699,7599063699,POINT (632664.601 4834122.468),traffic_signals,632664.600944,4.834122e+06
7599063700,7599063700,POINT (632694.101 4834100.709),traffic_signals,632694.101193,4.834101e+06
7599063701,7599063701,POINT (632534.744 4834249.312),traffic_signals,632534.744442,4.834249e+06


I am only interested in a few proximity features for my application, and the function to measure distance needs the projection in espg 4362 + the x and y coordinates named as x y.

In [92]:
stations_OldToronto_4326

,osmid,geometry,highway,x,y
20380905,20380905,POINT (-79.41168 43.71211),traffic_signals,-79.411685,43.712112
20380929,20380929,POINT (-79.39758 43.67721),traffic_signals,-79.397579,43.677207
20380931,20380931,POINT (-79.43902 43.66846),traffic_signals,-79.439018,43.668458
20380933,20380933,POINT (-79.45122 43.66574),traffic_signals,-79.451220,43.665739
20380934,20380934,POINT (-79.46378 43.66364),traffic_signals,-79.463781,43.663637
...,...,...,...,...,...
7598237115,7598237115,POINT (-79.38698 43.68947),stop,-79.386975,43.689471
7599063699,7599063699,POINT (-79.35499 43.64827),traffic_signals,-79.354992,43.648266
7599063700,7599063700,POINT (-79.35463 43.64807),traffic_signals,-79.354631,43.648065
7599063701,7599063701,POINT (-79.35657 43.64943),traffic_signals,-79.356570,43.649431


In [93]:
stations_OldToronto_md = stations_OldToronto_4326

# Generating proximity features for logistic regression/random forest

I was not able to find a function in python that allows me to easily measure proximity features, so I am going to use a function I found online. 

In [99]:
from sklearn.neighbors import BallTree
def get_nearest(src_points, candidates, k_neighbors=1):
    """Find nearest neighbors for all source points from a set of candidate points"""

    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=15, metric='haversine')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    # Get closest indices and distances (i.e. array at index 0)
    # note: for the second closest points, you would take index 1, etc.
    closest = indices[0]
    closest_dist = distances[0]

    # Return indices and distances
    return (closest, closest_dist)


def nearest_neighbor(left_gdf, right_gdf, return_dist=False):
    """
    For each point in left_gdf, find closest point in right GeoDataFrame and return them.

    NOTICE: Assumes that the input Points are in WGS84 projection (lat/lon).
    """

    left_geom_col = left_gdf.geometry.name
    right_geom_col = right_gdf.geometry.name

    # Ensure that index in right gdf is formed of sequential numbers
    right = right_gdf.copy().reset_index(drop=True)

    # Parse coordinates from points and insert them into a numpy array as RADIANS
    left_radians = numpy.array(
        left_gdf[left_geom_col].apply(lambda geom: (geom.x * numpy.pi / 180, geom.y * numpy.pi / 180)).to_list())
    right_radians = numpy.array(
        right[right_geom_col].apply(lambda geom: (geom.x * numpy.pi / 180, geom.y * numpy.pi / 180)).to_list())

    # Find the nearest points
    # -----------------------
    # closest ==> index in right_gdf that corresponds to the closest point
    # dist ==> distance between the nearest neighbors (in meters)

    closest, dist = get_nearest(src_points=left_radians, candidates=right_radians)

    # Return points from right GeoDataFrame that are closest to points in left GeoDataFrame
    closest_points = right.loc[closest]

    # Ensure that the index corresponds the one in left_gdf
    closest_points = closest_points.reset_index(drop=True)

    # Add distance if requested
    if return_dist:
        # Convert to meters from radians
        earth_radius = 6371000  # meters
        closest_points['distance'] = dist * earth_radius

    return closest_points

Find closest public transport stop for each building and get also the distance based on haversine distance.**Note**:haversine distance, which is implemented here is a bit slower than using e.g. 'euclidean' metric but useful as we get the distance between points in meters.

In [100]:
for i in features_location_names:
    robbery_backgroundpoints_gdf_4326[i] = nearest_neighbor(robbery_backgroundpoints_gdf_4326,
                                                            stations_OldToronto_md.loc[
                                                                stations_OldToronto_md['highway'] == i],
                                                            return_dist=True)['distance']
    robbery_points_gdf_4326[i] = nearest_neighbor(robbery_points_gdf_4326,
                                                  stations_OldToronto_md.loc[
                                                      stations_OldToronto_md['highway'] == i],
                                                  return_dist=True)['distance']

for i in features_location_names:
    pedestrian_backgroundpoints_gdf_4326[i] = nearest_neighbor(pedestrian_backgroundpoints_gdf_4326,
                                                            stations_OldToronto_md.loc[
                                                                stations_OldToronto_md['highway'] == i],
                                                            return_dist=True)['distance']
    pedestrian_points_gdf_4326[i] = nearest_neighbor(pedestrian_points_gdf_4326,
                                                  stations_OldToronto_md.loc[
                                                      stations_OldToronto_md['highway'] == i],
                                                  return_dist=True)['distance']
    



Adding some psedo data based on the ranges of what are in the occurrences

In [101]:
special = list(set(list(robbery_points_gdf_4326)) - set(list(robbery_backgroundpoints_gdf_4326)))
for i in special:
    longer = list(robbery_points_gdf_4326[i].unique())
    call = numpy.repeat(longer,
                        len(robbery_backgroundpoints_gdf_4326) / len(longer) * 2)
    shuffle(call)
    robbery_backgroundpoints_gdf_4326[i] = call[0:len(robbery_backgroundpoints_gdf_4326)]

robbery_points_gdf_4326['Presence_Absence'] = numpy.repeat(1,
                                                           len(robbery_points_gdf_4326))
robbery_backgroundpoints_gdf_4326['Presence_Absence'] = numpy.repeat(0,
                                                                     len(robbery_backgroundpoints_gdf_4326))

special = list(set(list(pedestrian_points_gdf_4326)) - set(list(pedestrian_backgroundpoints_gdf_4326)))
for i in special:
    longer = list(pedestrian_points_gdf_4326[i].unique())
    call = numpy.repeat(longer,
                        len(pedestrian_backgroundpoints_gdf_4326) / len(longer) * 2)
    shuffle(call)
    pedestrian_backgroundpoints_gdf_4326[i] = call[0:len(pedestrian_backgroundpoints_gdf_4326)]

pedestrian_points_gdf_4326['Presence_Absence'] = numpy.repeat(1,
                                                           len(pedestrian_points_gdf_4326))
pedestrian_backgroundpoints_gdf_4326['Presence_Absence'] = numpy.repeat(0,
                                                                     len(pedestrian_backgroundpoints_gdf_4326))

In [102]:
# now we can combine both data sets and
robbery_model_data = robbery_points_gdf_4326.append(
    pandas.DataFrame(data=robbery_backgroundpoints_gdf_4326),
    ignore_index=True)
robbery_model_data_final = robbery_model_data.iloc[:, numpy.r_[0, 2:len(list(robbery_model_data))]]
robbery_model_data_final.Presence_Absence.value_counts()



0    20564
1     5141
Name: Presence_Absence, dtype: int64

In [103]:
pedestrian_model_data = pedestrian_points_gdf_4326.append(
    pandas.DataFrame(data=pedestrian_backgroundpoints_gdf_4326),
    ignore_index=True)
pedestrian_model_data_final = pedestrian_model_data.iloc[:, numpy.r_[0, 2:len(list(pedestrian_model_data))]]
pedestrian_model_data_final.Presence_Absence.value_counts()

0    25732
1     6433
Name: Presence_Absence, dtype: int64

In [104]:
# need some dummy variables for my categorical data
# One-hot encode the data using pandas get_dummies
robbery_model_data_dummies = pandas.get_dummies(robbery_model_data_final)
pedestrian_model_data_dummies = pandas.get_dummies(pedestrian_model_data_final)
# NAs will give errors, so need to drop them
for i in list(robbery_model_data_dummies):
    robbery_model_data_dummies.dropna(subset=[i], inplace=True)
for i in list(pedestrian_model_data_dummies):
    pedestrian_model_data_dummies.dropna(subset=[i], inplace=True)

# do I need to balance data?
robbery_model_data_dummies.Presence_Absence.value_counts()
pedestrian_model_data_dummies.Presence_Absence.value_counts()

0    25732
1     6382
Name: Presence_Absence, dtype: int64

In [105]:
# combining the psedo absences and and occurences
pedestrian_model_data_dummies_p = pedestrian_model_data_dummies.loc[
    pedestrian_model_data_dummies["Presence_Absence"] == 1]
pedestrian_model_data_dummies_a = pedestrian_model_data_dummies.loc[
    pedestrian_model_data_dummies["Presence_Absence"] == 0]
pedestrian_model_data_dummies_a = pedestrian_model_data_dummies_a.sample(n=len(pedestrian_model_data_dummies_p))
pedestrian_model_data_dummies_bal = pedestrian_model_data_dummies_p.append(
    pandas.DataFrame(data=pedestrian_model_data_dummies_a),
    ignore_index=True)


robbery_model_data_dummies_p = robbery_model_data_dummies.loc[
    robbery_model_data_dummies["Presence_Absence"] == 1]
robbery_model_data_dummies_a = robbery_model_data_dummies.loc[
    robbery_model_data_dummies["Presence_Absence"] == 0]
robbery_model_data_dummies_a = robbery_model_data_dummies_a.sample(n=len(robbery_model_data_dummies_p))
robbery_model_data_dummies_bal = robbery_model_data_dummies_p.append(
    pandas.DataFrame(data=robbery_model_data_dummies_a),
    ignore_index=True)

# Saving Data for Predicting Probabilties

In [107]:
import pickle
robbery_model_data_dummies_bal.to_pickle("C:/Users/jodyn/Google Drive/Insight/Processed Data/robbery_model_data_dummies_bal.pkl")
pedestrian_model_data_dummies_bal.to_pickle("C:/Users/jodyn/Google Drive/Insight/Processed Data/pedestrian_model_data_dummies_bal.pkl")